In [ ]:
public abstract class Result
{

}

public class IntegerResult : Result
{
    public int Value {get;set;}
}

public class BetweenIntegerResult : Result
{
    public int From {get;set;}
    public int To {get;set;}
}
public class ListIntegerResult : Result
{
    public List<int> Ids {get;set;}
}

public enum Operator
{
    Power,
    Multiple,
    Sum
}

public interface IOperatorStrategy
{
    Result Process(string operatorValue);
}

public class IntegerProcessor : IOperatorStrategy
{
    public Result Process(string operatorValue)
    {
         int.TryParse(operatorValue, out int intValue);

         return new IntegerResult(){Value = intValue};
    }
}

public class BetweenIntegerProcessor : IOperatorStrategy
{
    public Result Process(string operatorValue)
    {
         string[] parts = operatorValue.Split(',');

         return new BetweenIntegerResult
        {
            From = int.Parse(parts[0]),
            To = int.Parse(parts[1])
        };
    }
}

public class ListIntegerProcessor : IOperatorStrategy
{
    public Result Process(string operatorValue)
    {
         return new ListIntegerResult
        {
            Ids = operatorValue.Split(',')
                               .Select(int.Parse)
                               .ToList()
        };
    }
}

public class ProcessorFactory
{
    public static IOperatorStrategy Create(Operator @operator)
    {
        return @operator switch
        {
            Operator.Power => new IntegerProcessor(),
            Operator.Multiple => new BetweenIntegerProcessor(),
            Operator.Sum => new ListIntegerProcessor(),
            _ => throw new NotImplementedException($"OperatorType {@operator} is not implemented")
        };
    }
}


public void Pattern(Operator @operator, string operatorValue)
{
    IOperatorStrategy strategy = ProcessorFactory.Create(@operator);

    Result result = strategy.Process(operatorValue); 

    if(result is IntegerResult intResult)
    {
        Console.WriteLine(intResult.Value);
    }

    if(result is BetweenIntegerResult betweenResult)
    {
        Console.WriteLine(betweenResult.From +" "+ betweenResult.To);
    }

    if(result is ListIntegerResult listResult)
    {
        foreach(var id in listResult.Ids)
            Console.WriteLine(id);
    }
}

//this way of pattern is 'Type Pattern'

public void PatternSwitch(Operator @operator, string operatorValue)
{
    IOperatorStrategy strategy = ProcessorFactory.Create(@operator);

    Result result = strategy.Process(operatorValue); 

    switch(result)
    {
        case IntegerResult intResult :
             Console.WriteLine(intResult.Value);
             break;
        case BetweenIntegerResult betweenResult :
             Console.WriteLine(betweenResult.From +" "+ betweenResult.To);
             break;
        default:
            break;
    }
}

### var Pattern

In [ ]:
bool IsJanetOrJohn (string name) 
{
    return  name.ToUpper() is var upper 
            && (upper == "JANET" || upper == "JOHN");
}
//This is equivalent to:

In [ ]:
bool IsJanetOrJohn (string name)
{
    string upper = name.ToUpper();
    return upper == "JANET" || upper == "JOHN";
}

### Constant Pattern  
The ***constant pattern*** lets you match directly to a constant, and is useful when
working with the `object` type

In [ ]:
void Foo (object obj)
{
    if (obj is 3) {}
}
//This expression in boldface is equivalent to the following:

In [ ]:
void Foo (object obj)
{
    if (obj is int && (int)obj == 3) {}
}

### Relational Patterns

In [ ]:
public void RelationIfPattern(int x)
{
    if (x is > 100) 
        Console.WriteLine ("x is greater than 100");
}

public string RelationSwitchPattern(decimal bmi)
{
    return bmi switch
    {
        < 18.5m => "underweight",
        < 25m => "normal",
        < 30m => "overweight",
        _ => "obese"
    };
}

### Pattern Combinators

In [ ]:
bool PrevIsJanetOrJohn (string name) 
{
    return  name.ToUpper() is var upper 
            && (upper == "JANET" || upper == "JOHN");
}

bool IsJanetOrJohn (string name) 
{
    return  name.ToUpper() is "JANET" or "JOHN";
}

bool IsVowel (char c) => c is 'a' or 'e' or 'i' or 'o' or 'u';

bool Between1And9 (int n) => n is >= 1 and <= 9;

bool IsLetter (char c) => c is >= 'a' and <= 'z'
                            or >= 'A' and <= 'Z';


### Tuple and Positional Patterns

In [ ]:
var p = (2, 3);
Console.WriteLine (p is (2, 3)); //true

enum Season { Spring, Summer, Fall, Winter };

int AverageCelsiusTemperature (Season season, bool daytime) =>
    (season, daytime) switch
    {
        (Season.Spring, true) => 20,
        (Season.Spring, false) => 16,
        (Season.Summer, true) => 27,
        (Season.Summer, false) => 22,
        (Season.Fall, true) => 18,
        (Season.Fall, false) => 12,
        (Season.Winter, true) => 10,
        (Season.Winter, false) => -2,
        _ => throw new Exception ("Unexpected combination")
    };


***positional pattern*** matches with `any type` that exposes a `Deconstruct` method

In [ ]:
record Point (int X, int Y);

var p = new Point (2, 2);
Console.WriteLine (p is (2, 2)); // True

Console.WriteLine (p is (var x, var y) && x == y); // True


string Print (object obj) => obj switch
{
    Point (0, 0) => "Empty point",
    Point (var x, var y) when x == y => "Diagonal", 
    _ => "else"
};

### property pattern

In [ ]:
//property pattern

public abstract class Result
{

}

public class IntegerResult : Result
{
    public int Value {get;set;}
}

var result = new IntegerResult(){Value = 5};

if(result is IntegerResult { Value : 8 } intResult)
{
    Console.WriteLine(intResult.Value);
}


bool ShouldAllow (Uri uri) => uri switch
{
    { Scheme: "http", Port: 80 } => true,
    { Scheme: "https", Port: 443 } => true,
    { Scheme: "ftp", Port: 21 } => true,
    { IsLoopback: true } => true,

    //nested property 
    { Scheme: { Length: 4 }, Port: 80 } => true,

    //from c#10 can be simplified
    //{ Scheme.Length: 4, Port: 80 } => true,

    //You can use other patterns inside property patterns, 
    //including the relational pattern:
    { Host: { Length: < 1000 }, Port: > 0 } => true,

    //More elaborate conditions can be expressed with a when clause:
    { Scheme: "http" } when string.IsNullOrWhiteSpace (uri.Query) => true,

    // first check properties then use them
    Uri{ Scheme: "http", Port: 88 } httpUri => httpUri.Host =="niazerooz.com",

    _ => false
};

bool ShouldAllow1 (Uri uri) => uri switch
{
    { Scheme: "http", Port: 80, Host: var host } => host.Length < 1000,
    { Scheme: "https", Port: 443 } => true,
    { Scheme: "ftp", Port: 21 } => true,
    { IsLoopback: true } => true,
    _ => false
};